# Texas College Review
Author: Mr. Frederick Orndorff
Date: January 26, 2020

## Code Documentation:
Most of the code below is from the Coursera labs within the __IBM Data Science__ Specialization.  I have modified the code to support my business problem.

## Introduction
__Business Problem:__
There are over 5,300 Colleges and Universities in the United States and over 450 of these colleges are in the state of Texas.  There are so many colleges in the Texas that a person could spend months (or even years) attempting to determine the best fit for a student -- or to determine which college is the best in Texas.  There are many personal opinions on which college ranks first, but can we use data to determine the best college in Texas?  

For this project I will use public web data and the Foursquare API to provide a ranking for some Texas colleges.  This will be a data driven solution ot the business problem of too many colleges to determine the best fit.

__Stakeholders:__
There are many stakeholders in this problem, including: students, parents (who might pay for the student to attend college), transfer students, as well as, the college administration and presidents.

## Data
__Map Data:__
I plan to create a choropeth map to provide a visual representation of crime and other data.  In order to adequately display the map I wanted to overlay the Texas counties on the state map.  The Texas state government provided the data used for the county overlays, located [at this link.](https://data.texas.gov/dataset/Texas-Counties-Centroid_Map/ups3-9e8m).  This dataset also provided the latitude and longitude coordinates needed to use the Foursquare API data.

__Informational Datasets:___
In addition to the county information I also found the following datasets that I will incooporate in our data driven solutions:

1. [2018 Texas Crime Data](http://www.dps.texas.gov/administration/crime_records/pages/crimestatistics.htm)

The Texas Department of Public safety compiles data from state and local law enforcement agencies across the country, as well as the FBI. The Interstate Identification Index is a cooperative program between the FBI and the states to facilitate the exchange of information among those states and the FBI in order to reduce duplication of effort in records retention and demands accurate information for success. Crime data is focused on violent crimes, hate crimes, and assaults.

2. [Texas State Expenditures by County](https://data.texas.gov/Government-and-Taxes/Texas-State-Expenditures-by-County/f2iw-dtqt)

Texas State Expenditures by County shows where state dollars are spent. It lists state expenditures by agency and type of expenditure in each county. This data provides an overview of where the State is spending the local tax dollars for improvement(s).

3. [Mixed Beverage Sales](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893)

This file contains a list of taxpayers required to report mixed beverage gross receipts tax reports under Texas law.  It provides total sales reported and taxes paid.

4. [List of Texas Colleges](http://www.txhighereddata.org/Interactive/Institutions.cfm)

This database provides the list of Colleges, Universities, and trade schools available in the state of Texas.

## Data Application to the Business Problem

__Assumptions___
I will make the following assumptions for the stakeholders:
1. Lower crime is better
2. Higher state expenditures in the county is better
3. Larger beverage sales is better (work hard - play harder!!)
4. Larger number of Foursquare data (venues) in the area is better.

## Methodology  

## Python packages used for this project:

In [ ]:
import os, pandas as pd, geocoder, requests, json, folium, geopy
import matplotlib.cm as cm, matplotlib.colors as colors, matplotlib.pyplot as plt
import numpy as np
from geopy.geocoders import Nominatim

I used some older python packages for this project since I was familiar with the correct syntax of the packages.  However, some of the packages resulted in 'warnings' from Python -- to use the updated package.  The below code will temporarily turnoff these warnings.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Set working directoy
#os.chdir("/Users/christinaorndorff/Documents/Fred's Stuff/Coursa/IBM_Capstone") #for MAC
os.chdir("C:/Users/OrndorFH01/Coursera/IBM_Capstone")

## Exploratory Data Analysis
__Texas County Map Data:__
There are 254 counties in Texas, therefore, the Texas county data has 254 rows of data.  Each row contains the latitude and longitude coordinates, the county name, the county number, the Federal Information Processing Standards code (provided by the U.S. census), the size of the county (including the land area), and the county location (which is the lat/long coordinates together).

In [ ]:
df = pd.read_csv('Texas_Counties_Centroid_Map.csv')
df.head()

I was only interested in four columns: latitude, longitude, county name, and county number.  All other columns were removed during data cleaning.  I kept the county number since another data base only contains the county number vice the county name, therefore, I need to keep the county number to adequately merge the two dataframes (this occurs in a later step).  Additionally, we had to make the county name lowercase to match the 'case' in other databases.

One major issue I found in the database was the incorrect labeling of the latitude and longitude coordinates.  For some reason the two coordinates were backwards (swapped).  I initially did not see this error, until I attempted to visualize the data on a folium map and it did not render correcly.  After many attempts and some in depth analysis - I determined the coordinates were backwards.  Once corrected - everything worked correctly.

In [ ]:
df1 = df.iloc[:, [0, 1, 2, 3]]
#for some reason the base .csv file has the Lat/Long values swapped -- we can fix this by renaming the columns
df1.columns = ['Longitude', 'Latitude', 'County_Name', 'County_Number']
#we will need to make the 'County_Name' lowercase -- to merge dfs later
df1['County_Name'] = df1['County_Name'].str.lower()
df1 = df1[['County_Name', 'County_Number', 'Latitude', 'Longitude']]

In [ ]:
df1.head()

In [ ]:
df1.shape #254 counties

__Mixed Beverage Sales:__
The mixed beverage sales includes over two-million rows and 24 columns of data.  This is a larger file - 453MB.  

Some interesting data points include: Location County, Liquor Receipts, Wine Receipts, Beer Receipts, and Total Receipts.  Focusing on these five columns I determined the most important data will be the location county and the total receipts.  

In [ ]:
df2 = pd.read_csv('Mixed_Beverage_Gross_Receipts.csv')
df2 = df2[['Location County', 'Total Receipts']]
#rename df2 columns
df2.columns = ['County_Number', 'Total_Alcohol_Sales']

Two small issues arose during inital data analysis.  The first issue was that each reporting venue within the county had a seperate data entry (therefore the 2 million entries).  This created the need to sum all the venues within each county - resulting in 254 unique 'total receipts', one for each county.  Secondly, the 'Location County' was a numerical entry - not text - therefore I needed to merge this dataframe using the county number.  Fortunately, the Texas County Map Data included the county number, so we can easly merge these two dataframes.  I orginally stripped the county number from the county database, but once I conducted some data analysis on the mixed beverage sales - I added the county number back so I could merege these databases.

In [ ]:
#sum 'Total_Alcohol_Sales' by 'County' -- using the reset_index() function keeps the output as a pandas df
df2_sum = df2.groupby('County_Number')['Total_Alcohol_Sales'].sum().reset_index()
df2_sum.head()

__First Merged Dataframe.__ 
To ensure the dataframes are compatible (there is a column to merge against), I went ahead and merged the first two dataframes and checked for consistency.  It works!!

In [ ]:
df_final = pd.merge(df1, df2_sum, on = "County_Number")
df_final.head()

__Texas State Expenditures by Country.__ 
The Texas state expenditures by country data includes over 88 thousand rows and 6 columns of data. While it is not as big as the mixed beverage sales database - there are some similarities.  Most noticeable is the multiple row entries for each county.  This issue requires me to combine entries on the county to determine a total amount spent.

In [ ]:
df3 = pd.read_csv('Texas_State_Expenditures_by_County.csv')

#only keep county and amount columns
df3 = df3[['County', 'Amount']]

#rename columns
df3.columns = ['County_Name', 'Expenditure_Amount']

#sum the row amounts by county
df3_sum = df3.groupby('County_Name')['Expenditure_Amount'].sum().reset_index()

This database created one new issue.  The county names were provided in all uppercase, which did not match our inital database of counties (Texas_Counties_Centroid_Map).  In order to have like columns to merge against - I had to make all the county names lowercase.

In [ ]:
#need to make the 'County_Name' lowercase so we can merge the two dfs
df3_sum['County_Name'] = df3_sum['County_Name'].str.lower()

Once this step was complete, I merged this dataframe with first merged dataframe.

In [ ]:
#add the 'Expenditure_Amount' to the final df
df_final = pd.merge(df_final, df3_sum, on = "County_Name")

df_final.head()

Later in the project I created a map with tags.  Unfortunately, I did not like the county displayed in all lower case, so I went bact to the orginal database and added the county names with the correct capitalization (County_Name2).

In [ ]:
df4 = df[['CNTY_NM', 'CNTY_NBR']]
df4.columns = ['County_Name2', 'County_Number']
df_final = pd.merge(df_final, df4, on = "County_Number")
df_final.head()

__[Texas Crime Data:](http://www.dps.texas.gov/administration/crime_records/pages/crimestatistics.htm)__
While I was researching the background for this topic - best colleges in Texas - I discovered that a major determining factor for college attendance is the crime rate for the surrounding area.  Therefore, I thought the first step in the project would be to create a choropleth map of Texas.  In order to complete this task, I needed another database with county crime data.  This section shows the steps I took to clean and modify the database to use for the choropleth map.

In [ ]:
df5 = pd.read_csv('TX_Crime2.csv')

#sum incidents across agency name and ensure the incident counts are integers
df5_sum = df5.groupby('Agency_Name')['Number_of_Incidents'].sum().reset_index()
df5_sum['Number_of_Incidents'] = df5_sum['Number_of_Incidents'].astype(int)

#the agency names are really county names - let's rename this column
df5_sum.columns = ['County_Name', 'Number_of_Incidents']

#need to make the 'County_Name' lowercase so we can merge the two dfs
df5_sum['County_Name'] = df5_sum['County_Name'].str.lower()
df5_sum['County_Name'] = df5_sum['County_Name'].str.title()
df5_sum.head()

## Visualizations (Maps and Charts)
This section starts to provide a visualization of the data to allow the user the ability to better understand the data.

First, lets determine the coordinates of Texas so we can center our maps correctly.

In [ ]:
address = 'Texas'

geolocator = Nominatim(user_agent = "texas_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Texas are {}, {}.'.format(latitude, longitude))

### Folium Map 1 (with labels)
Using our final database I created my first visualization using a folium map.  This map provided the locations of all 254 counties for Texas.  Initially, the county labels were all provided in lowercase - which I did not like.  This is where I decieded to reach back to the orginal database and rescrape the county names with the correct capitalization.

_NOTE: the graphics might not render in github (a known error) -- so I attached the maps as screenshots in the base capstone folder on github._

In [ ]:
map_texas = folium.Map(location = [latitude, longitude], zoom_start = 6)

for lat, lng, county_name, total_alcohol_sales, expenditure_amount in zip(df_final['Latitude'], 
                          df_final['Longitude'],
                          df_final['County_Name2'], 
                          df_final['Total_Alcohol_Sales'],
                          df_final['Expenditure_Amount']):
    label = '{}'.format(county_name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_texas)

map_texas

Well, the map looks like polka-dots imposed over a map of Texas.  This really does not provide enough context to start an analysis.  Lets look at other options, starting with geojson data.

### Geojson data map
Using geojson data allowed me to create a map with county boundries superimposed on a map of Texas.  For my first couple of tries (multiple hours of work) the county boundries resulted in grey shading.  What I determined was the geojson data had the 'county' feature with different capitalization.  Going back to my Texas crime data - I had to complete additional data analysis to create a useful database (with correct county tags).

Another issue I determined was that the Chrome web browser does not support a folium map with a large number of features.  I had to revert to FireFox (or Safari) for the map to display...this was another multiple hour dilemma.

The map provided below, shows the crime data for the State of Texas in 2018.

_NOTE: the graphics might not render in github (a known error) -- so I attached the maps as screenshots in the base capstone folder on github._

In [ ]:
texas_geo = r'tx_counties.geojson'

import numpy as np

texas_map2 = folium.Map(location = [latitude, longitude], zoom_start = 6)

threshold_scale = np.linspace(start = df5_sum['Number_of_Incidents'].min(),
                              stop = df5_sum['Number_of_Incidents'].max(),
                              num = 10)
threshold_scale = threshold_scale.tolist()
threshold_scale[-1] = threshold_scale[-1] + 1

texas_map2.choropleth(geo_data = texas_geo,
                      data = df5_sum,
                      columns=['County_Name', 'Number_of_Incidents'],
                      key_on='feature.properties.COUNTY',
                      threshold_scale = threshold_scale,
                      fill_color='YlOrRd',
                      fill_opacity = 0.7,
                      line_opacity = 1,
                      legend_name = 'Crime in Texas'
                     )

texas_map2

__Better Map Data__
This map is much better, but we still do not know which counties have the highest crime.  Maybe if we merge the two maps we can provide a better visualization.

In [ ]:
for lat, lng, county_name, total_alcohol_sales, expenditure_amount in zip(df_final['Latitude'], 
                          df_final['Longitude'],
                          df_final['County_Name2'], 
                          df_final['Total_Alcohol_Sales'],
                          df_final['Expenditure_Amount']):
    label = '{}'.format(county_name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(texas_map2)

texas_map2

It looks like there are five counties with the highest crime rates for 2018.

1. Harris County (near Houston)
2. Dallas County
3. Bexar County (near San Antonio)
4. Tarrant County (near Dallas)
5. Travis County (near Austin)

There are also a number of counties with no reported criminal incidents.  I will have to look into these counties to ensure there are no issues with the reporting.  These counties could be sparsely populated with little to no crime - but I will have to check past reporting to make sure.

### Folium Map 2 (Mixed Beverage Sales)
Is the crime data linked to the total mixed beverage sales?  In other words does increased alcohol sales increase the crime rate in college towns?  I will create another choropleth map to provide a good visualization.  

__First Issue__
There is a large variance in the amount of alcohol sales.  The choropleth map does not like this variance when determining the correct 'bins' to place counties in.  First, lets do a quick scatter plot to double-check the outliers.

In [ ]:
#I'll only use the data we need for this plot
df_plot = df_final[['County_Number', 'Total_Alcohol_Sales']]
df_plot['County_Number'] = df_plot['County_Number'].astype(int)

In [ ]:
df_plot.plot(kind = 'scatter',
             x = 'County_Number',
             y = 'Total_Alcohol_Sales',
             figsize = (15,10))
plt.show()

From the plot we can see there are a majority (>50%) of sales between 0 and 10 million dollars.  However, there are still approximately 15% of sales much higher than the 10 million dollar threshold.  Therefore, we should manually set the threshold values for the choropleth map to control with the high variance of the data.

I start with subsetting the 'total alcohol sales' into 9 equal bins.  Then create a choropleth on these new bins

In [ ]:
x = np.linspace(start = 0, stop = df_final['Total_Alcohol_Sales'].max(), num = 9)
x = x.tolist()
x

__Second Issue__

Well, we have a problem!! The county names in the final_df does not match the geojson data and our county names in our df are different...so lets merge dataframes on the first five letters of the county name.  This should provide a working df for the choropleth map.

In [ ]:
#create a new column with the first 5 letters of the county name
df_final['new_col'] = df_final['County_Name2'].astype(str).str[0:4]
df5_sum['new_col'] = df5_sum['County_Name'].astype(str).str[0:4]

#merge the two df
merged_left = pd.merge(left = df_final,
                       right = df5_sum, 
                       how = 'left', 
                       left_on = 'new_col', 
                       right_on = 'new_col')

Now back to the choropleth graph.

In [ ]:
texas_map3 = folium.Map(location = [latitude, longitude], zoom_start = 6)

texas_map3.choropleth(geo_data = texas_geo,
                      data = merged_left,
                      columns=['County_Name_y', 'Total_Alcohol_Sales'],
                      key_on='feature.properties.COUNTY',
                      threshold_scale = x,
                      bins=x, 
                      fill_color='Blues',
                      fill_opacity = 0.7,
                      line_opacity = 0.2,
                      legend_name = 'Alcohol Sales in Texas'
                     )

texas_map3

This map does not provide much data to work with.  Lets modify the bins and see what this does.

In [ ]:
conditions = [
    (merged_left['Total_Alcohol_Sales'] >= 0) & (merged_left['Total_Alcohol_Sales'] <= x[1]),
    (merged_left['Total_Alcohol_Sales'] > x[1]) & (merged_left['Total_Alcohol_Sales'] <= x[2]),
    (merged_left['Total_Alcohol_Sales'] > x[2]) & (merged_left['Total_Alcohol_Sales'] <= x[3]),
    (merged_left['Total_Alcohol_Sales'] > x[3]) & (merged_left['Total_Alcohol_Sales'] <= x[4]),
    (merged_left['Total_Alcohol_Sales'] > x[4]) & (merged_left['Total_Alcohol_Sales'] <= x[5]),
    (merged_left['Total_Alcohol_Sales'] > x[5]) & (merged_left['Total_Alcohol_Sales'] <= x[6]),
    (merged_left['Total_Alcohol_Sales'] > x[6]) & (merged_left['Total_Alcohol_Sales'] <= x[7]),
    (merged_left['Total_Alcohol_Sales'] > x[7]) & (merged_left['Total_Alcohol_Sales'] <= x[8])
]
choices = ['1', '2', '3', '4', '5', '6', '7', '8']
merged_left['map_data'] = np.select(conditions, choices, default = '0')
merged_left['map_data'] = merged_left['map_data'].astype(int)
merged_left.head()

In [ ]:
texas_map4 = folium.Map(location = [latitude, longitude], zoom_start = 6)

threshold_scale = np.linspace(start = merged_left['map_data'].min(),
                              stop = merged_left['map_data'].max(),
                              num = 9)
threshold_scale = threshold_scale.astype(int).tolist()
threshold_scale[-1] = threshold_scale[-1] + 1

texas_map4.choropleth(geo_data = texas_geo,
                      data = merged_left,
                      columns=['County_Name_y', 'map_data'],
                      key_on='feature.properties.COUNTY',
                      threshold_scale = threshold_scale,
                      fill_color='Blues',
                      fill_opacity = 0.7,
                      line_opacity = 0.2,
                      legend_name = 'Alcohol Sales in Texas'
                     )

texas_map4

In [ ]:
for lat, lng, county_name, total_alcohol_sales, expenditure_amount in zip(df_final['Latitude'], 
                          df_final['Longitude'],
                          df_final['County_Name2'], 
                          df_final['Total_Alcohol_Sales'],
                          df_final['Expenditure_Amount']):
    label = '{}'.format(county_name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(texas_map4)

texas_map4

It looks like the counties with the highest alcohol sales include:

1. Hardin County (near Beaumont)
2. Dallas County
3. Travis County (near Austin)
4. Tarrant County (near Dallas)
5. Bexar County (near San Antonio)

There seems to be some correlation with the alcohol sales and the crime rate.  Four of the top five crime areas are also in the top five for alcohol sales.  However, Harris county the top crime area does not rank in the top alcohol sales.  Harris county is actually numer 83 on the alcohol sales list, maybe something else is causing the high crime rate in this county.

### Folium Map 3 (Texas State Expenditures)

The last dataset I looked at was the Texas state expenditures.  This dataset shows the tax dollars the Texas Government invested in counties.  One could assume the Texas state Government would invest in counties they feel are the most important to the state of Texas.  Therefore, I expect the higher amount of investment equates to a 'better' county.

In this section we create the last folium map to visualize the total expenditures by county.

__First Issue.__ 
This seems to be a reoccurring issue - the variance in the values are causing issues when numpy creates the threshold bins.  Lets see how different the data is.

In [ ]:
df_plot = merged_left[['County_Number', 'Expenditure_Amount']]
df_plot['County_Number'] = df_plot['County_Number'].astype(int)

In [ ]:
df_plot.plot(kind = 'scatter',
             x = 'County_Number',
             y = 'Expenditure_Amount',
             figsize = (15,10))
plt.show()

From the scatter plot we can see the majority of the data falls within 0 to 2.5 million dollars.  However, the largest expenditure amount is much higher - almost 10 times higher - therefore I will create better bins for the choropleth map.

In [ ]:
x = np.linspace(start = 0, stop = df_final['Expenditure_Amount'].max(), num = 9)
x = x.tolist()
x

In [ ]:
conditions2 = [
    (merged_left['Expenditure_Amount'] >= 0) & (merged_left['Expenditure_Amount'] <= x[1]),
    (merged_left['Expenditure_Amount'] > x[1]) & (merged_left['Expenditure_Amount'] <= x[2]),
    (merged_left['Expenditure_Amount'] > x[2]) & (merged_left['Expenditure_Amount'] <= x[3]),
    (merged_left['Expenditure_Amount'] > x[3]) & (merged_left['Expenditure_Amount'] <= x[4]),
    (merged_left['Expenditure_Amount'] > x[4]) & (merged_left['Expenditure_Amount'] <= x[5]),
    (merged_left['Expenditure_Amount'] > x[5]) & (merged_left['Expenditure_Amount'] <= x[6]),
    (merged_left['Expenditure_Amount'] > x[6]) & (merged_left['Expenditure_Amount'] <= x[7]),
    (merged_left['Expenditure_Amount'] > x[7]) & (merged_left['Expenditure_Amount'] <= x[8]),
]
choices = ['1', '2', '3', '4', '5', '6', '7', '8']
merged_left['map_data2'] = np.select(conditions2, choices, default = '0')
merged_left['map_data2'] = merged_left['map_data2'].astype(int)
merged_left.head()

In [ ]:
texas_map5 = folium.Map(location = [latitude, longitude], zoom_start = 6)

threshold_scale = np.linspace(start = merged_left['map_data2'].min().astype(int),
                              stop = merged_left['map_data2'].max().astype(int),
                              num = 9)
threshold_scale = threshold_scale.astype(int).tolist()
threshold_scale[-1] = threshold_scale[-1] + 1

texas_map5.choropleth(geo_data = texas_geo,
                      data = merged_left,
                      columns=['County_Name_y', 'map_data2'],
                      key_on='feature.properties.COUNTY',
                      threshold_scale = threshold_scale,
                      fill_color='Greens',
                      fill_opacity = 0.7,
                      line_opacity = 0.2,
                      legend_name = 'Expenditure Amount by County in Texas'
                     )



texas_map5

In [ ]:
texas_map5 = folium.Map(location = [latitude, longitude], zoom_start = 6)

threshold_scale = np.linspace(start = merged_left['map_data2'].min().astype(int),
                              stop = merged_left['map_data2'].max().astype(int),
                              num = 9)
threshold_scale = threshold_scale.astype(int).tolist()
threshold_scale[-1] = threshold_scale[-1] + 1

texas_map5.choropleth(geo_data = texas_geo,
                      data = merged_left,
                      columns=['County_Name_y', 'map_data2'],
                      key_on='feature.properties.COUNTY',
                      threshold_scale = threshold_scale,
                      fill_color='Greens',
                      fill_opacity = 0.7,
                      line_opacity = 0.2,
                      legend_name = 'Expenditure Amount by County in Texas'
                     )

for lat, lng, county_name, total_alcohol_sales, expenditure_amount in zip(df_final['Latitude'], 
                          df_final['Longitude'],
                          df_final['County_Name2'], 
                          df_final['Total_Alcohol_Sales'],
                          df_final['Expenditure_Amount']):
    label = '{}'.format(county_name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(texas_map5)
    
    texas_map5

It looks like the counties with the highest state expenditures include:

1. Travis County (near Austin)
2. Harris County
3. Dallas County
4. Bexar County (near San Antonio)
5. Tarrant County (near Dallas)

The top counties in Texas seem to be rather constant, with four counties falling within the top 5 in all areas.  However, let's link this analysis to the top colleges we would like to focus the analysis on.

## College Analysis
For this section, I focused the analysis on the following colleges:
* University of Texas (UT) - Travis County
* University of Texas at San Antonio (UTSA) - Bexar County
* University of Houston (Houston) - Harris County
* Texas A&M (TAMU) - Brazos County
* Texas Tech University (TTU) - Lubbock County

These five colleges are the ones I am most intersted in attending.  If a stakeholder has a different list - it is easy to change, since we have the data for every county in Texas.    
First I will subset the final dataframe to only include the data that corresponds to the list of colleges above.

In [ ]:
college_df = merged_left[merged_left.County_Name2.isin(['Bexar', 
                                                        'Brazos', 
                                                        'Harris', 
                                                        'Lubbock', 
                                                        'Travis']) & 
                         merged_left.County_Name_y.isin(['Bexar County',
                                                         'Brazos County', 
                                                         'Harris County', 
                                                         'Lubbock County', 
                                                         'Travis County'])]

In [ ]:
df_merged = pd.merge(left = college_df,
                       right = df5_sum, 
                       how = 'left', 
                       left_on = 'County_Name_y', 
                       right_on = 'County_Name')

In [ ]:
df_merged = df_merged[['County_Name', 
                       'Number_of_Incidents_y',
                      'Total_Alcohol_Sales',
                      'Expenditure_Amount']]
df_merged.columns = ['County_Name',
                    'Crime_Rate',
                    'Alcohol_Sales',
                    'Expenditures']

I added the rankings between the five colleges we picked.

In [ ]:
df_merged['Crime_Rate_Rank'] = df_merged['Crime_Rate'].rank(ascending = True)
df_merged['Alcohol_Sales_Rank'] = df_merged['Alcohol_Sales'].rank(ascending = False)
df_merged['Expenditures_Rank'] = df_merged['Expenditures'].rank(ascending = False)
df_merged.insert(0, 'College', ['UT', 'TAMU', 'UTSA', 'TTU', 'Houston'])

Lets look at the total rankings.

In [ ]:
df_ranked = df_merged[['College',
                       'Crime_Rate_Rank',
                       'Alcohol_Sales_Rank',
                       'Expenditures_Rank']]
df_ranked['Total'] = df_ranked.sum(axis=1)
df_ranked = df_ranked.sort_values(by=['Total'])

In [ ]:
df_ranked

## Results
It looks like the __University of Texas__ is the clear winner; with the University of Texas at San Antonio and Texas Tech University in a tie for second place.  So now that we have a college to attend, where is the best place to live?  For the next part of the analysis, I will utilize the Foursquare API data and k-means clustering to determine the best location to live within Travis County – where the University of Texas is located.


# Foursquare API Analysis  

 

In [ ]:
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

## Travis County Base Map: 

In [ ]:
address = 'Travis County, Texas'

geolocator = Nominatim(user_agent = "texas_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Travis County are {}, {}.'.format(latitude, longitude))

In [ ]:
os.chdir("C:/Users/OrndorFH01/Coursera/IBM_Capstone/Austin")

In [ ]:
zip_df = pd.read_csv('Texas_latitude_longitude.csv')
zip_df = zip_df[zip_df['City'].str.contains("Austin")]
zip_df.rename(columns = {'Zip':'Zip_Code'}, inplace = True) 
zip_df.head()

In [ ]:
austin_zip = pd.read_csv('Austin_County.csv')
austin_zip2 = austin_zip.loc[austin_zip['County'] == 'Travis']
austin_zip2['Population'] = austin_zip2['Population'].astype(int)
austin_zip2.head()

In [ ]:
austin_merge = pd.merge(austin_zip2, zip_df, on = "Zip_Code")

In [ ]:
austin_merge.head()

There are a number of zipcodes that were created for Post Office boxes with an assigned population of zero.  I will go ahead and delete these 'unique' zipcodes.

In [ ]:
indexZip = austin_merge[austin_merge['Population'] == 0].index
austin_merge.drop(indexZip, inplace = True)
austin_merge['Zip_Code'] = austin_merge['Zip_Code'].astype(str)

In [ ]:
austin_merge.head()

In [ ]:
austin_merge = austin_merge.reset_index(drop=True)

In [ ]:
austin_merge.shape

In [ ]:
austin_merge.dtypes

In [ ]:
austin_texas = folium.Map(location = [latitude, longitude], zoom_start = 10.5)

for lat, lng, zip_code, pop in zip(austin_merge['Latitude'], 
                          austin_merge['Longitude'],
                          austin_merge['Zip_Code'], 
                          austin_merge['Population']):
    label = 'Zip Code:{}\nPopulation:{}'.format(zip_code, pop)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(austin_texas)

austin_texas

## Foursquare Analysis

In [ ]:
CLIENT_ID = 'GBTVHG14LE2WKWDFJYDTF4UTKWLDY5SAULYKHNHS5DVORM5A' # your Foursquare ID
CLIENT_SECRET = 'KWLNACCQGFTFG01N1530FPPZCGGTES4VK4AW3ZH4QDLIQU44' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

__Venues in Travis County.__  
To determine the best neighborhood to live in, I will first explore the nearby venues for each zip code in Travis County.

In [ ]:
neighborhood_latitude = austin_merge.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = austin_merge.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = austin_merge.loc[0, 'Zip_Code'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 250 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=250, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip_Code', 
                  'Zip_Code Latitude', 
                  'Zip_Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
austin_venues = getNearbyVenues(names = austin_merge['Zip_Code'],
                                latitudes = austin_merge['Latitude'],
                                longitudes = austin_merge['Longitude'])

In [ ]:
print(austin_venues.shape)
austin_venues.head()

In [ ]:
print('There are {} uniques categories.'.format(len(austin_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
austin_onehot = pd.get_dummies(austin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
austin_onehot['Zip_Code'] = austin_venues['Zip_Code'] 

# move neighborhood column to the first column
fixed_columns = [austin_onehot.columns[-1]] + list(austin_onehot.columns[:-1])
austin_onehot = austin_onehot[fixed_columns]

austin_onehot.head()

In [ ]:
austin_grouped = austin_onehot.groupby('Zip_Code').mean().reset_index()
austin_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip_Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Zip_Code_venues_sorted = pd.DataFrame(columns=columns)
Zip_Code_venues_sorted['Zip_Code'] = austin_grouped['Zip_Code']

for ind in np.arange(austin_grouped.shape[0]):
    Zip_Code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(austin_grouped.iloc[ind, :], num_top_venues)

Zip_Code_venues_sorted.head()

## KMeans Clustering

In [ ]:
# set number of clusters
kclusters = 5

austin_grouped_clustering = austin_grouped.drop('Zip_Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(austin_grouped_clustering)

# check cluster labels generated for the first 10 rows in the dataframe
kmeans.labels_[0:10] 

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
distortions = []
k = range (1,5)
for n in k:
    kmeanModel = KMeans(n_clusters = n, random_state = 0).fit(austin_grouped_clustering)
    distortions.append(sum(np.min(cdist(austin_grouped_clustering, 
                                        kmeanModel.cluster_centers_,
                                        'canberra'), 
                                  axis = 1))/austin_grouped_clustering.shape[0])

In [ ]:
plt.plot(k, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('Optimal Cluster Number(s)')
plt.show()

In [ ]:
# add clustering labels
austin_merged = austin_merge
Zip_Code_venues_sorted['Cluster_Labels'] = kmeans.labels_
austin_merged = austin_merged.join(Zip_Code_venues_sorted.set_index('Zip_Code'), 
                               on='Zip_Code')
austin_merged.head() # check the last columns!

## Visualizations (Maps and Charts)

__Source Code Documentation.__  I found the code for the bar chart (below) at: https://github.com/Srcanyildiz/istanbul/blob/master/Istanbul.ipynb

In [ ]:
count_venue = austin_merged
count_venue = count_venue.drop(['Zip_Code', 'Type', 'County', 'Population', 'City', 'State',
                               'Latitude', 'Timezone', 'Daylight savings time flag', 'geopoint'],
                              axis = 1)
count_venue = count_venue.groupby(['Cluster_Labels', '1st Most Common Venue']).size().reset_index(name = 'Counts')

cv_cluster = count_venue.pivot(index='Cluster_Labels', columns = '1st Most Common Venue', values = 'Counts')
cv_cluster = cv_cluster.fillna(0).astype(int).reset_index(drop = True)

frame = cv_cluster.plot(kind = 'bar', figsize = (20,8), width = 0.8)

plt.legend(labels = cv_cluster.columns, fontsize = 14)
plt.title("Number of Venues in Each Cluster", fontsize = 16)
plt.xticks(fontsize = 14, rotation = 0)
plt.xlabel("Number of Venues", fontsize = 14)
plt.ylabel("Clusters", fontsize = 14)

In [ ]:
plt.show()

In [ ]:
null_columns = austin_merged.columns[austin_merged.isnull().any()]
print(austin_merged[austin_merged.isnull().any(axis=1)][null_columns].head())

In [ ]:
austin_merged = austin_merged.dropna()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(austin_merged['Latitude'], 
                                  austin_merged['Longitude'], 
                                  austin_merged['Zip_Code'], 
                                  austin_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
austin_merged2 = austin_merged[['Zip_Code', '1st Most Common Venue', '2nd Most Common Venue', 
                                '3rd Most Common Venue', 'Cluster_Labels']]


In [ ]:
austin_merged2

## Results  

### Part 1: County Review  
For the first part of the project, I used public data to determine the top five Texas counties in three areas.  The results of the analysis are provided below:

Counties with the highest __crime rates for 2018__
1. Harris County (near Houston)
2. Dallas County
3. Bexar County (near San Antonio)
4. Tarrant County (near Dallas)
5. Travis County (near Austin)

Counties with the highest __alcohol sales__
1. Hardin County (near Beaumont)
2. Dallas County
3. Travis County (near Austin)
4. Tarrant County (near Dallas)
5. Bexar County (near San Antonio)

Counties with the highest __expenditures__ (State Investments)
1. Travis County (near Austin)
2. Harris County
3. Dallas County
4. Bexar County (near San Antonio)
5. Tarrant County (near Dallas)

### Part 2: College Review
Using the results of our Texas county analysis, I next ranked the five colleges I was most interested in attending.
* University of Texas (UT) - Travis County
* University of Texas at San Antonio (UTSA) - Bexar County
* University of Houston (Houston) - Harris County
* Texas A&M (TAMU) - Brazos County
* Texas Tech University (TTU) - Lubbock County

In [ ]:
df_ranked

The rankings showed that the University of Texas at Austin was a clear winner, placing third on crime and first in alcohol sales and expenditures.  

### Part 3: Foursquare API
Now that we have a college picked and the county ranked -- I turned to the Foursquare API data to better understand my new college town.  Using the Foursquare API data I saw a majority of the county includes venues that cater to college aged people -- coffee shops and fast food restaurants.  

The town was easily split into 5 clusters using K means clustering.  The four clusters includes one with yoga; one with coffe shops and resturants; one with transportation hubs; one with recreaction; and lastly one with a golf driving range (TopGolf!!).

## Observations  
* There is a lot of public data provided on the internet.  However, the data provided is not clean.  A majority of the time spent on the project was doing initial data cleaning and analysis
* Not all programs are created equally.  Understanding the limitations of programs allows for a faster completion time and less stress.  During this project I determined Chrome is not adequate for choropleth maps with large json data files
* Visualizations were key to better understanding the data
* Austin or Travis County was first in alcohol sales, but none of the clusters focused on bars or other drinking establishments - this seems a little strange
* Further analysis is needed to determine the lack of data for some smaller counties
* Texas is huge and has a very large income and expenditure rate  

## Conclusion
Data driven decisions provides another tool to decision making processes.  The vast amounts of data allows data scientists the ability to provide another viewpoint -- other than the opinion based decisions we make daily.  From this project we can see that stakeholders have many schools to choose from and using data provides the first step in focusing the options for decision makers.  This was a great learning experience and it showed me the importance of data in decision making.